In [2]:
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from datetime import datetime
import distutils.dir_util
from keras.callbacks import CSVLogger
from keras.utils.np_utils import to_categorical
import numpy as np

from keras.layers import Dense, Input, Lambda, BatchNormalization, Conv2D, Dropout
from keras.layers import ZeroPadding2D, MaxPooling2D, AveragePooling2D, Activation, Flatten
from keras.layers import GlobalAveragePooling2D
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

from keras import __version__
print(__version__)

2.0.3


In [3]:
base_model = VGG16(weights='imagenet', include_top=False)

In [35]:
# get data for visualisation later

from keras.preprocessing.image import ImageDataGenerator
# def preprocess_input(x, data_format=None):
#     """Preprocesses a tensor encoding a batch of images.
#     # Arguments
#         x: input Numpy tensor, 4D.
#         data_format: data format of the image tensor.
#     # Returns
#         Preprocessed tensor.
#     """

#     # 'RGB'->'BGR'
#     x = x[:, :, ::-1]
#     # Zero-center by mean pixel
#     x[:, :, 0] -= 103.939
#     x[:, :, 1] -= 116.779
#     x[:, :, 2] -= 123.68
#     return x

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

#batch_size = 32

train_generator = train_datagen.flow_from_directory(
        '/home/ubuntu/data/fishing/train',
        target_size=(224, 224),
        batch_size=8,
        #batch_size=batch_size,
        shuffle=False,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        '/home/ubuntu/data/fishing/valid',
        target_size=(224, 224),
        shuffle=False,
        #batch_size=batch_size,
        batch_size=10,
        class_mode='categorical')

Found 3276 images belonging to 8 classes.
Found 500 images belonging to 8 classes.


In [36]:
def create_precomputed_data(model, batches):
    filenames = batches.filenames
    conv_features = model.predict_generator(batches, (batches.samples/batches.batch_size ))
    labels_onehot = to_categorical(batches.classes)
    labels = batches.classes
    return (filenames, conv_features, labels_onehot, labels)

In [37]:
3276/8

409

In [ ]:
## NB was having trouble visualising the resnet conv layer, unsure if the dimensions are suitable, so 
# came back to vgg - trying here to precompute the convolution layers for vgg but getting OOM error, reduced the batch 
# size and checked that nothing was trainable - that shouldn't matter just for prediction though.. 

In [38]:
# base_model.layers.pop()
layers = base_model.layers
for l in layers:
    l.trainable = False

In [39]:
trn_filenames, trn_conv_features, trn_labels, trn_labels_1 = create_precomputed_data(base_model, train_generator)
val_filenames, val_conv_features, val_labels, val_labels_1 = create_precomputed_data(base_model, validation_generator)

ResourceExhaustedError: OOM when allocating tensor with shape[8,3,224,224]
	 [[Node: block1_conv1/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](_recv_input_1_0/_105, block1_conv1/kernel/read)]]
	 [[Node: block5_pool/MaxPool/_107 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_20_block5_pool/MaxPool", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'block1_conv1/convolution', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_ins

In [31]:
base_model.input_shape=(3,224,224)

AttributeError: can't set attribute